In [3]:
from src.Game import Game
from src.Players.Minesweeper_bot import Minesweeper_bot
from src.UI.GUI_Bot_Inputs import GUI_Bot_Inputs
from src.UI.No_UI import No_UI
from src.Grid import Grid

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from tqdm.notebook import tqdm
import numpy as np
from scipy.stats import beta as beta_law
import matplotlib.pyplot as plt

import torch
from random import sample

from models.Game_Tensor_Interface import Game_Tensor_Interface

In [4]:
grid_size = 8
mine_percent = 0.15

# Uniform sampling

In [4]:
#Uniform sampling

n_game = 10000
sample_per_game = 3
sample_last = True

tensor_list = []
mines_list =[]
tensor_interface = Game_Tensor_Interface()

for i in tqdm(range(n_game)):
    # Grid initialisation
    grid = Grid(grid_size, grid_size, mine_percent)
    game = Game(grid)
    bot = Minesweeper_bot()
    game.action(*bot.action(*game.visible_grid()))

    visible_grid_list = []
    while not game.is_ended():
        # Register every grids
        visible_grid = game.visible_grid()
        visible_grid_list.append(visible_grid)

        # Play the game
        game.action(*bot.action(*visible_grid))

    if len(visible_grid_list) <= sample_per_game:
        selection = visible_grid_list
    elif sample_last:
        selection = sample(visible_grid_list[:-1], sample_per_game-1)
        selection.append(visible_grid_list[-1])
    else:
        selection = sample(visible_grid_list, sample_per_game)

    for grid_values, grid_view in selection:
        tensor_list.append(tensor_interface.to_tensor(grid_values, grid_view))
        mines_list.append(grid.mines)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
mines_tensor = torch.tensor(mines_list)

C:\Users\toine\AppData\Local\Temp\ipykernel_12232\465619920.py:1: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  mines_tensor = torch.tensor(mines_list)
C:\Users\toine\AppData\Local\Temp\ipykernel_12232\465619920.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  mines_tensor = torch.tensor(mines_list)


In [10]:
data = torch.stack(tensor_list).type(torch.uint8)

In [12]:
data.shape

torch.Size([29868, 10, 8, 8])

In [13]:
mines_tensor.shape

torch.Size([29868, 8, 8])

In [14]:
torch.save([data, mines_tensor], 'dataset/uniform_bot/8x8_2990.pt')

# Losing position sampling

In [5]:
grid_size = 12
mine_percent = 0.22

In [7]:
n_game = 10

def gather_data(n_game:int):
    tensor_list = []
    mines_list =[]
    tensor_interface = Game_Tensor_Interface()

    for i in tqdm(range(n_game)):
        # Grid initialisation
        grid = Grid(grid_size, grid_size, mine_percent)
        game = Game(grid)
        bot = Minesweeper_bot()
        game.action(*bot.action(*game.visible_grid()))

        # Play until we are about to do a mistake (or we win)
        next_action = bot.action(*game.visible_grid())
        while not grid.mines[*next_action] and not game.is_ended():
            game.action(*next_action)
            if not game.is_ended():
                next_action = bot.action(*game.visible_grid())
        
        if grid.mines[*next_action]:
            tensor_list.append(tensor_interface.to_tensor(*game.visible_grid()))
            mines_list.append(grid.mines)

    data = torch.stack(tensor_list).type(torch.uint8)
    mines_tensor = torch.tensor(mines_list)
    return data, mines_tensor


In [8]:
all_res = [gather_data(x) for x in [10]*5]

  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\toine\AppData\Local\Temp\ipykernel_8264\4046932697.py:27: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  mines_tensor = torch.tensor(mines_list)
C:\Users\toine\AppData\Local\Temp\ipykernel_8264\4046932697.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  mines_tensor = torch.tensor(mines_list)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
tensor_representations = torch.cat([x for x, y in all_res])
mines_tensors = torch.tensor(np.concatenate([y for x, y in all_res]))

In [20]:
mines_tensors.shape

torch.Size([27, 12, 12])

In [17]:
torch.save([tensor_representations, mines_tensors], f'dataset/lose_bot/12x12_{len(tensor_representations)}.pt')